<h1> Text Classification using TensorFlow/Keras on Cloud ML Engine </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for Machine Learning using BigQuery
<li> Creating a text classification model using the Estimator API with a Keras model
<li> Training on Cloud ML Engine
<li> Deploying the model
<li> Predicting with model
<li> Rerun with pre-trained embedding
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-23ec85502280dc5d'
PROJECT = 'qwiklabs-gcp-23ec85502280dc5d'
REGION = 'us-east1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.8'

if 'COLAB_GPU' in os.environ:  # this is always set on Colab, the value is 0 or 1 depending on whether a GPU is attached
  from google.colab import auth
  auth.authenticate_user()
  # download "sidecar files" since on Colab, this notebook will be on Drive
  !rm -rf txtclsmodel
  !git clone --depth 1 https://github.com/GoogleCloudPlatform/training-data-analyst
  !mv  training-data-analyst/courses/machine_learning/deepdive/09_sequence/txtclsmodel/ .
  !rm -rf training-data-analyst
  # downgrade TensorFlow to the version this notebook has been tested with
  !pip install --upgrade tensorflow==$TFVERSION

In [3]:
import tensorflow as tf
print(tf.__version__)

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


We will look at the titles of articles and figure out whether the article came from the New York Times, TechCrunch or GitHub. 

We will use [hacker news](https://news.ycombinator.com/) as our data source. It is an aggregator that displays tech related headlines from various  sources.

### Creating Dataset from BigQuery 

Hacker news headlines are available as a BigQuery public dataset. The [dataset](https://bigquery.cloud.google.com/table/bigquery-public-data:hacker_news.stories?tab=details) contains all headlines from the sites inception in October 2006 until October 2015. 

Here is a sample of the dataset:

In [4]:
%%load_ext google.cloud.bigquery

In [5]:
%%bigquery --project $PROJECT
SELECT
  url, title, score
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  LENGTH(title) > 10
  AND score > 10
  AND LENGTH(url) > 0
LIMIT 10

url  \
0  https://www.kickstarter.com/projects/carlosxcl...   
1  http://vancouver.en.craigslist.ca/van/roo/2035...   
2       https://github.com/Groundworkstech/Submicron   
3                          http://empowerunited.com/   
4  http://themanufacturingrevolution.com/braun-vs...   
5                     https://github.com/styleguide/   
6  http://zoomzum.com/10-best-firefox-add-ons-to-...   
7                              http://www.zintin.com   
8           http://founderdating.com/comingtodinner/   
9  http://tech.matchfwd.com/poor-mans-template-ab...   

                                               title  score  
0  Show HN: Code Cards, Like Texas hold 'em for p...     11  
1                              Best Roommate Ad Ever     11  
2                           Deep-Submicron Backdoors     11  
3            Could this be the solution for the 99%?     11  
4  Braun vs. Apple: Is copying designs theft or i...     11  
5            GitHub Styleguide - CSS, HTML, JS, Ruby     11  
6  Essential Firefox Add-Ons to Make You More Pro...     11  
7            Feedback on our social media iPhone app     11  
8   Guess Who’s Coming to Dinner…To Save Our Company     11  
9        Poor Man's Template A/B Testing (in Django)     11

Let's do some regular expression parsing in BigQuery to get the source of the newspaper article from the URL. For example, if the url is http://mobile.nytimes.com/...., I want to be left with <i>nytimes</i>

In [6]:
%%bigquery --project $PROJECT
SELECT
  ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
  COUNT(title) AS num_articles
FROM
  `bigquery-public-data.hacker_news.stories`
WHERE
  REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
  AND LENGTH(title) > 10
GROUP BY
  source
ORDER BY num_articles DESC
LIMIT 10

source  num_articles
0     blogspot         41386
1       github         36525
2   techcrunch         30891
3      youtube         30848
4      nytimes         28787
5       medium         18422
6       google         18235
7    wordpress         17667
8  arstechnica         13749
9        wired         12841

Now that we have good parsing of the URL to get the source, let's put together a dataset of source and titles. This will be our labeled dataset for machine learning.

In [7]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

query="""
SELECT source, LOWER(REGEXP_REPLACE(title, '[^a-zA-Z0-9 $.-]', ' ')) AS title FROM
  (SELECT
    ARRAY_REVERSE(SPLIT(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.'))[OFFSET(1)] AS source,
    title
  FROM
    `bigquery-public-data.hacker_news.stories`
  WHERE
    REGEXP_CONTAINS(REGEXP_EXTRACT(url, '.*://(.[^/]+)/'), '.com$')
    AND LENGTH(title) > 10
  )
WHERE (source = 'github' OR source = 'nytimes' OR source = 'techcrunch')
"""

df = bq.query(query + " LIMIT 5").to_dataframe()
df.head()

source                                              title
0  github                              php bdd is now  nice 
1  github                       mpv video player 0.2 release
2  github  show hn  re-thinking the business card with dr...
3  github           update css js from chrome developer tool
4  github  simple way to start development with flask usi...

For ML training, we will need to split our dataset into training and evaluation datasets (and perhaps an independent test dataset if we are going to do model or feature selection based on the evaluation dataset).  

A simple, repeatable way to do this is to use the hash of a well-distributed column in our data (See https://www.oreilly.com/learning/repeatable-sampling-of-data-sets-in-bigquery-for-machine-learning).

In [8]:
traindf = bq.query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) > 0").to_dataframe()
evaldf  = bq.query(query + " AND MOD(ABS(FARM_FINGERPRINT(title)),4) = 0").to_dataframe()

Below we can see that roughly 75% of the data is used for training, and 25% for evaluation. 

We can also see that within each dataset, the classes are roughly balanced.

In [9]:
traindf['source'].value_counts()

github        27445
techcrunch    23131
nytimes       21586
Name: source, dtype: int64

In [10]:
evaldf['source'].value_counts()

github        9080
techcrunch    7760
nytimes       7201
Name: source, dtype: int64

Finally we will save our data, which is currently in-memory, to disk.

In [11]:
import os, shutil
DATADIR='data/txtcls'
shutil.rmtree(DATADIR, ignore_errors=True)
os.makedirs(DATADIR)
traindf.to_csv( os.path.join(DATADIR,'train.tsv'), header=False, index=False, encoding='utf-8', sep='\t')
evaldf.to_csv( os.path.join(DATADIR,'eval.tsv'), header=False, index=False, encoding='utf-8', sep='\t')

In [12]:
!head -3 data/txtcls/train.tsv

github	feminist-software-foundation complains about reddit on a github pull request
github	expose sps as web services on the fly.
github	show hn  linuxexplorer


In [13]:
!wc -l data/txtcls/*.tsv

  24041 data/txtcls/eval.tsv
  72162 data/txtcls/train.tsv
  96203 total


### TensorFlow/Keras Code

Please explore the code in this <a href="txtclsmodel/trainer">directory</a>: `model.py` contains the TensorFlow model and `task.py` parses command line arguments and launches off the training job.

In particular look for the following:

1. [tf.keras.preprocessing.text.Tokenizer.fit_on_texts()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#fit_on_texts) to generate a mapping from our word vocabulary to integers
2. [tf.keras.preprocessing.text.Tokenizer.texts_to_sequences()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences) to encode our sentences into a sequence of their respective word-integers
3. [tf.keras.preprocessing.sequence.pad_sequences()](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences) to pad all sequences to be the same length

The embedding layer in the keras model takes care of one-hot encoding these integers and learning a dense emedding represetation from them. 

Finally we pass the embedded text representation through a CNN model pictured below

<img src=images/txtcls_model.png  width=25%>

### Run Locally (optional step)
Let's make sure the code compiles by running locally for a fraction of an epoch.
This may not work if you don't have all the packages installed locally for gcloud (such as in Colab).
This is an optional step; move on to training on the cloud.

In [14]:
%%bash
source activate py2env  # gcloud uses python2 by default
## Make sure we have the latest version of Google Cloud Storage package
pip install --upgrade google-cloud-storage
rm -rf txtcls_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/txtclsmodel/trainer \
   -- \
   --output_dir=${PWD}/txtcls_trained \
   --train_data_path=${PWD}/data/txtcls/train.tsv \
   --eval_data_path=${PWD}/data/txtcls/eval.tsv \
   --num_epochs=0.1

  Found existing installation: google-api-core 0.1.4
    Uninstalling google-api-core-0.1.4:
      Successfully uninstalled google-api-core-0.1.4
  Found existing installation: google-cloud-core 0.28.1
    Uninstalling google-cloud-core-0.28.1:
      Successfully uninstalled google-cloud-core-0.28.1


google-cloud-monitoring 0.28.0 has requirement google-api-core<0.2.0dev,>=0.1.1, but you'll have google-api-core 1.8.2 which is incompatible.
google-cloud-monitoring 0.28.0 has requirement google-cloud-core<0.29dev,>=0.28.0, but you'll have google-cloud-core 0.29.1 which is incompatible.
google-cloud-bigquery 0.23.0 has requirement google-cloud-core<0.24dev,>=0.23.1, but you'll have google-cloud-core 0.29.1 which is incompatible.
/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:TF_CONFIG environment variable: {u'environment': u'cloud', u'cluster': {}, u'job': {u'args': [u'--output_dir=/content/datalab/training-data-analyst/courses/machine_learning/deepdive/09_sequence/txtcls_trained', u'--train_data_path=/co

### Train on the Cloud

Let's first copy our training data to the cloud:

In [15]:
%%bash
gsutil cp data/txtcls/*.tsv gs://${BUCKET}/txtcls/

Copying file://data/txtcls/eval.tsv [Content-Type=text/tab-separated-values]...
Copying file://data/txtcls/train.tsv [Content-Type=text/tab-separated-values]...
-
Operation completed over 2 objects/5.4 MiB.                                      


In [16]:
%%bash
OUTDIR=gs://${BUCKET}/txtcls/trained_fromscratch
JOBNAME=txtcls_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
 --region=$REGION \
 --module-name=trainer.task \
 --package-path=${PWD}/txtclsmodel/trainer \
 --job-dir=$OUTDIR \
 --scale-tier=BASIC_GPU \
 --runtime-version=$TFVERSION \
 -- \
 --output_dir=$OUTDIR \
 --train_data_path=gs://${BUCKET}/txtcls/train.tsv \
 --eval_data_path=gs://${BUCKET}/txtcls/eval.tsv \
 --num_epochs=5

jobId: txtcls_190402_205838
state: QUEUED


CommandException: 1 files/objects could not be removed.
Job [txtcls_190402_205838] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe txtcls_190402_205838

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs txtcls_190402_205838


Change the job name appropriately. View the job in the console, and wait until the job is complete.

In [18]:
!gcloud ml-engine jobs describe txtcls_190402_205838

createTime: '2019-04-02T20:58:43Z'
etag: 5H3pglTRctQ=
jobId: txtcls_190402_205838
state: PREPARING
trainingInput:
  args:
  - --output_dir=gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch
  - --train_data_path=gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/train.tsv
  - --eval_data_path=gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/eval.tsv
  - --num_epochs=5
  jobDir: gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch
  packageUris:
  - gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/packages/4f8feb586e1fa71f2efc514d90a711361aecd60e9710f623139632c180a10495/text_classification-1.0.tar.gz
  pythonModule: trainer.task
  region: us-east1
  runtimeVersion: '1.8'
  scaleTier: BASIC_GPU
trainingOutput: {}

View job in the Cloud Console at:
https://console.cloud.google.com/ml/jobs/txtcls_190402_205838?project=qwiklabs-gcp-23ec85502280dc5d

View logs at:
https://console.cloud.google.com/logs?resource=ml.googleapis.com%2Fjob_id%2Ftxtcls_190402_205838&project=qwiklab

### Monitor training with TensorBoard

If Tensorboard appears blank, try refreshing after a few minutes.
If you removed the directory that Tensorboard was monitoring, stop and restart Tensorboard.

In [19]:
from google.datalab.ml import TensorBoard
TensorBoard().start('gs://{}/txtcls/trained_fromscratch'.format(BUCKET))

TensorBoard was started successfully with pid 4420. Click here to access it.

4420

In [20]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 4420


### Results
What accuracy did you get? You should see around 80%.

### Deploy trained model 

Once your training completes you will see your exported models in the output directory specified in Google Cloud Storage. 

You should see one model for each training checkpoint (default is every 1000 steps).

In [21]:
%%bash
gsutil ls gs://${BUCKET}/txtcls/trained_fromscratch/export/exporter/

gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/
gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/1554239009/
gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/1554239034/
gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/1554239056/
gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/1554239078/
gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/1554239100/
gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/1554239122/
gs://qwiklabs-gcp-23ec85502280dc5d/txtcls/trained_fromscratch/export/exporter/1554239140/


We will take the last export and deploy it as a REST API using Google Cloud Machine Learning Engine 

In [22]:
%%bash
MODEL_NAME="txtcls"
MODEL_VERSION="v1_fromscratch"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/txtcls/trained_fromscratch/export/exporter/ | tail -1)
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME} --quiet
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version=$TFVERSION

Created ml engine model [projects/qwiklabs-gcp-23ec85502280dc5d/models/txtcls].
Creating version (this might take a few minutes)......
...............................................................................................................................................................................................................................................................................done.


### Get Predictions

Here are some actual hacker news headlines gathered from July 2018. These titles were not part of the training or evaluation datasets.

In [23]:
techcrunch=[
  'Uber shuts down self-driving trucks unit',
  'Grover raises €37M Series A to offer latest tech products as a subscription',
  'Tech companies can now bid on the Pentagon’s $10B cloud contract'
]
nytimes=[
  '‘Lopping,’ ‘Tips’ and the ‘Z-List’: Bias Lawsuit Explores Harvard’s Admissions',
  'A $3B Plan to Turn Hoover Dam into a Giant Battery',
  'A MeToo Reckoning in China’s Workplace Amid Wave of Accusations'
]
github=[
  'Show HN: Moon – 3kb JavaScript UI compiler',
  'Show HN: Hello, a CLI tool for managing social media',
  'Firefox Nightly added support for time-travel debugging'
]

Our serving input function expects the already tokenized representations of the headlines, so we do that pre-processing in the code before calling the REST API.

Note: Ideally we would do these transformation in the tensorflow graph directly instead of relying on separate client pre-processing code (see: [training-serving skew](https://developers.google.com/machine-learning/guides/rules-of-ml/#training_serving_skew)), howevever the pre-processing functions we're using are python functions so cannot be embedded in a tensorflow graph. 

See the <a href="text_classification_native.ipynb">text_classification_native</a> notebook for a solution to this.

In [24]:
import pickle
from tensorflow.python.keras.preprocessing import sequence
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

requests = techcrunch+nytimes+github

# Tokenize and pad sentences using same mapping used in the deployed model
tokenizer = pickle.load( open( "txtclsmodel/tokenizer.pickled", "rb" ) )

requests_tokenized = tokenizer.texts_to_sequences(requests)
requests_tokenized = sequence.pad_sequences(requests_tokenized,maxlen=50)

# JSON format the requests
request_data = {'instances':requests_tokenized.tolist()}

# Authenticate and call CMLE prediction API 
credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

parent = 'projects/%s/models/%s' % (PROJECT, 'txtcls') #version is not specified so uses default
response = api.projects().predict(body=request_data, name=parent).execute()

# Format and print response
for i in range(len(requests)):
  print('\n{}'.format(requests[i]))
  print(' github    : {}'.format(response['predictions'][i]['dense_1'][0]))
  print(' nytimes   : {}'.format(response['predictions'][i]['dense_1'][1]))
  print(' techcrunch: {}'.format(response['predictions'][i]['dense_1'][2]))


Uber shuts down self-driving trucks unit
 github    : 0.00017399985517840832
 nytimes   : 0.11583155393600464
 techcrunch: 0.883994460105896

Grover raises €37M Series A to offer latest tech products as a subscription
 github    : 1.962046383141569e-07
 nytimes   : 0.0165999922901392
 techcrunch: 0.9833998084068298

Tech companies can now bid on the Pentagon’s $10B cloud contract
 github    : 6.184969970490783e-05
 nytimes   : 0.29176780581474304
 techcrunch: 0.7081703543663025

‘Lopping,’ ‘Tips’ and the ‘Z-List’: Bias Lawsuit Explores Harvard’s Admissions
 github    : 0.0012872441438958049
 nytimes   : 0.7943981885910034
 techcrunch: 0.20431455969810486

A $3B Plan to Turn Hoover Dam into a Giant Battery
 github    : 0.0038961044047027826
 nytimes   : 0.9670146107673645
 techcrunch: 0.02908925525844097

A MeToo Reckoning in China’s Workplace Amid Wave of Accusations
 github    : 0.0005226038629189134
 nytimes   : 0.9927476048469543
 techcrunch: 0.006729775574058294

Show HN: Moon – 3

How many of your predictions were correct?

### Rerun with Pre-trained Embedding

In the previous model we trained our word embedding from scratch. Often times we get better performance and/or converge faster by leveraging a pre-trained embedding. This is a similar concept to transfer learning during image classification.

We will use the popular GloVe embedding which is trained on Wikipedia as well as various news sources like the New York Times.

You can read more about Glove at the project homepage: https://nlp.stanford.edu/projects/glove/

You can download the embedding files directly from the stanford.edu site, but we've rehosted it in a GCS bucket for faster download speed.

In [36]:
!gsutil cp gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt gs://$BUCKET/txtcls/

Copying gs://cloud-training-demos/courses/machine_learning/deepdive/09_sequence/text_classification/glove.6B.200d.txt [Content-Type=text/plain]...
- [1 files][661.3 MiB/661.3 MiB]      0.0 B/s                                   
Operation completed over 1 objects/661.3 MiB.                                    


Once the embedding is downloaded re-run your cloud training job with the added command line argument: 

` --embedding_path=gs://${BUCKET}/txtcls/glove.6B.200d.txt`

Be sure to change your OUTDIR so it doesn't overwrite the previous model.

While the final accuracy may not change significantly, you should notice the model is able to converge to it much more quickly because it no longer has to learn an embedding from scratch.

#### References
- This implementation is based on code from: https://github.com/google/eng-edu/tree/master/ml/guides/text_classification.
- See the full text classification tutorial at: https://developers.google.com/machine-learning/guides/text-classification/
-

## Next step
Client-side tokenizing in Python is hugely problematic. See <a href="text_classification_native.ipynb">Text classification with native serving</a> for how to carry out the preprocessing in the serving function itself.

Copyright 2017 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License